<a href="https://colab.research.google.com/github/PrachiSinghal86/Introduction-to-Pytorch/blob/master/SaveAndLoad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import helper
from torchvision import datasets,transforms



In [0]:
import fc_model

ModuleNotFoundError: ignored

In [0]:
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,)),])
#load trainset
trainset=datasets.FashionMNIST('Fashion-MNIST_data/',download=True,train=True,transform=transform)
trainloader=torch.utils.data.DataLoader(trainset,batch_size=64,shuffle=True)
#load test set
testset=datasets.FashionMNIST('Fashion-MNIST_data/',download=True,train=False,transform=transform)
testloader=torch.utils.data.DataLoader(testset,batch_size=64,shuffle=True)

0it [00:00, ?it/s]

26427392it [00:07, 3422795.86it/s]                              


Extracting Fashion-MNIST_data/FashionMNIST/raw/train-images-idx3-ubyte.gz to Fashion-MNIST_data/FashionMNIST/raw


0it [00:00, ?it/s]

32768it [00:00, 72465.36it/s]                            
0it [00:00, ?it/s]

Extracting Fashion-MNIST_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to Fashion-MNIST_data/FashionMNIST/raw


4423680it [00:01, 3025735.14it/s]                            
0it [00:00, ?it/s]

Extracting Fashion-MNIST_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to Fashion-MNIST_data/FashionMNIST/raw


8192it [00:00, 27114.22it/s]            


Extracting Fashion-MNIST_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to Fashion-MNIST_data/FashionMNIST/raw
Processing...
Done!


In [0]:
images,labels=next(iter(trainloader))

In [0]:
model =nn.Sequential(nn.Linear(784,256),nn.Dropout(p=0.2),
                     nn.ReLU(),nn.Linear(256,128),
                     nn.Dropout(p=0.2),nn.ReLU(),nn.Linear(128,64),
                     
                     nn.Dropout(p=0.2),nn.ReLU(),nn.Linear(64,10),nn.LogSoftmax(dim=1))
crite=nn.NLLLoss() 
optimizer=optim.Adam(model.parameters(),lr=0.01)



In [0]:
from torch import optim
test_losses,train_losses=[],[]
epoch=16
optimizer=optim.Adam(model.parameters(),lr=0.02)
for i in range(epoch):
  runlos=0
  for images,labels in trainloader:
    images=images.view(images.shape[0],-1)   #flatten the images
    optimizer.zero_grad()
    output=model.forward(images)
    loss=crite(output,labels)
    loss.backward()
    optimizer.step()
    runlos+=loss.item()
  else:
    testloss=0
    acc=0
    with torch.no_grad():
      model.eval()
      for images,labels in trainloader:
        images=images.view(images.shape[0],-1) 
        logps=model(images)
        testloss+=crite(logps,labels)
        ps=torch.exp(logps)
        topp,topclass=ps.topk(1,dim=1)
        equals= topclass==labels.view(*topclass.shape)
        acc+=torch.mean(equals.type(torch.FloatTensor))
    model.train()

In [0]:
print("Model state dictionary",model.state_dict().keys())
print(model)

Model state dictionary odict_keys(['0.weight', '0.bias', '3.weight', '3.bias', '6.weight', '6.bias', '9.weight', '9.bias'])
Sequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): Dropout(p=0.2, inplace=False)
  (2): ReLU()
  (3): Linear(in_features=256, out_features=128, bias=True)
  (4): Dropout(p=0.2, inplace=False)
  (5): ReLU()
  (6): Linear(in_features=128, out_features=64, bias=True)
  (7): Dropout(p=0.2, inplace=False)
  (8): ReLU()
  (9): Linear(in_features=64, out_features=10, bias=True)
  (10): LogSoftmax()
)


In [0]:
torch.save(model.state_dict(),'checkpoint.pth')

In [0]:
state_dict=torch.load('checkpoint.pth')
print(model.state_dict().keys())

odict_keys(['0.weight', '0.bias', '3.weight', '3.bias', '6.weight', '6.bias', '9.weight', '9.bias'])


In [0]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [0]:
checkpoint= {'input_size':784,
            'output_size':10,
             'hiden_layers':[256,128,64],
             'state_dict':model.state_dict()
             }
torch.save(checkpoint,'checkpoint.pth')